# Initialize

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
pd.options.display.max_columns=None
from src.client.client import ClientHelper
from src.analysis.analyse import OrdersAnalyser, AssetAnalyser
from src.utils.utils import load_api_keys_from_json

In [ ]:
api_key, api_secret = load_api_keys_from_json('../api_key.json')
client_helper = ClientHelper(api_key, api_secret)

# Analyze portfolio

In [ ]:
prices = client_helper.query_prices()
history_assets = client_helper.get_history_assets('SPOT', days=30)
asset_analyser = AssetAnalyser(client_helper)
portfolio_fig = asset_analyser.plot_asset_composition_in_usdt(history_assets, prices)
portfolio_fig

In [ ]:
asset_history_fig = asset_analyser.plot_asset_usdt_value_history(history_assets)
asset_history_fig

# Analyze orders history

In [ ]:
orders = client_helper.get_all_orders()
orders.head()

In [ ]:
order_analyser = OrdersAnalyser(client_helper, orders)
mean_price = order_analyser.calculate_mean_price()
mean_price = pd.merge(mean_price, prices.rename(columns={'price':'current_price'}), on=['base_coin','quote_coin'], how='left')
mean_price['price_change_usd'] = mean_price['current_price'] - mean_price['average_price']
mean_price['price_change_percent'] = mean_price['price_change_usd']/mean_price['average_price']*100
display(mean_price.round(3))

In [ ]:
for base_coin in ['CELR', 'ETH']:
    price_history = client_helper.get_historical_prices(base_coin+'USDT', start_date='1 Jan, 2021')
    fig = order_analyser.plot_transactions(base_coin, price_history)
    fig.show()

# Trade

In [ ]:

# get market depth
depth = client.get_order_book(symbol='BNBBTC')

# place a test market buy order, to place an actual order use the create_order function
order = client.create_test_order(
    symbol='BNBBTC',
    side=Client.SIDE_BUY,
    type=Client.ORDER_TYPE_MARKET,
    quantity=100)

# get all symbol prices
prices = client.get_all_tickers()

# withdraw 100 ETH
# check docs for assumptions around withdrawals
from binance.exceptions import BinanceAPIException
try:
    result = client.withdraw(
        asset='ETH',
        address='<eth_address>',
        amount=100)
except BinanceAPIException as e:
    print(e)
else:
    print("Success")

# fetch list of withdrawals
withdraws = client.get_withdraw_history()

# fetch list of ETH withdrawals
eth_withdraws = client.get_withdraw_history(coin='ETH')

# get a deposit address for BTC
address = client.get_deposit_address(coin='BTC')

# get historical kline data from any date range

# fetch 1 minute klines for the last day up until now
klines = client.get_historical_klines("BNBBTC", Client.KLINE_INTERVAL_1MINUTE, "1 day ago UTC")

# fetch 30 minute klines for the last month of 2017
klines = client.get_historical_klines("ETHBTC", Client.KLINE_INTERVAL_30MINUTE, "1 Dec, 2017", "1 Jan, 2018")

# fetch weekly klines since it listed
klines = client.get_historical_klines("NEOBTC", Client.KLINE_INTERVAL_1WEEK, "1 Jan, 2017")

# socket manager using threads
twm = ThreadedWebsocketManager()
twm.start()

# depth cache manager using threads
dcm = ThreadedDepthCacheManager()
dcm.start()

def handle_socket_message(msg):
    print(f"message type: {msg['e']}")
    print(msg)

def handle_dcm_message(depth_cache):
    print(f"symbol {depth_cache.symbol}")
    print("top 5 bids")
    print(depth_cache.get_bids()[:5])
    print("top 5 asks")
    print(depth_cache.get_asks()[:5])
    print("last update time {}".format(depth_cache.update_time))

twm.start_kline_socket(callback=handle_socket_message, symbol='BNBBTC')

dcm.start_depth_cache(callback=handle_dcm_message, symbol='ETHBTC')

# replace with a current options symbol
options_symbol = 'BTC-210430-36000-C'
dcm.start_options_depth_cache(callback=handle_dcm_message, symbol=options_symbol)

# join the threaded managers to the main thread
twm.join()
dcm.join()